In [33]:
# Primeiro carregue as imagens compactadas no notebook do collab
# Autorize o Collab a acessar seus arquivos no drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [34]:
import numpy as np
import os

# Imports for neural network
from keras.layers import Conv2D, UpSampling2D, InputLayer, Conv2DTranspose
from keras.layers import Activation, Dense, Dropout, Flatten
from keras.layers.normalization import BatchNormalization
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from sklearn.model_selection import train_test_split
import tensorflow as tf

# Imports for image processing
from skimage.color import rgb2lab, lab2rgb, rgb2gray, xyz2lab
import imageio
import cv2

# Leitura do Dataset e Pré-processamento das Imagens

In [ ]:
imagePath = '/content/drive/MyDrive/dataset/landscape Images/color/'
X = []
Y = []

i=0
for filename in os.listdir(imagePath):
    i = i + 1

    image = imageio.imread(imagePath+filename)
    # Alterando dimensao da imagem
    image = cv2.resize(image, dsize=(256, 256), interpolation=cv2.INTER_CUBIC)
    image = np.array(img_to_array(image), dtype=float)

    # Conversao de espaço de cores para Lab
    # X contem canal L e Y os canais de cor a e b
    varX = rgb2lab(1.0/255*image)[:,:,0]
    varX = varX.reshape(1, 256, 256, 1)
    varY = rgb2lab(1.0/255*image)[:,:,1:]
    varY /= 128
    varY = varY.reshape(1, 256, 256, 2)

    X.append(varX)
    Y.append(varY)

    if i >= 1000:
        break

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.5, random_state=1)

X_train = np.array(X_train)
y_train = np.array(y_train)
X_train = np.array(X_test)
y_train = np.array(y_test)

# Definição e treinamento do Modelo de Redes Neurais

In [ ]:
# Definição do Modelo
model = Sequential()
model.add(InputLayer(input_shape=(256, 256, 1)))
model.add(Conv2D(8, (3, 3), activation='relu', padding='same', strides=2))
model.add(Conv2D(8, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(16, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(16, (3, 3), activation='relu', padding='same', strides=2))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same', strides=2))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(16, (3, 3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(2, (3, 3), activation='tanh', padding='same'))

model.compile(optimizer='rmsprop',loss='mse')

In [ ]:
# Treinamento do Modelo
def GenerateInputs(X_,y_):
    for i in range(len(X_)):
        X_input = X_[i].reshape(1,256,256,1)
        y_input = y_[i].reshape(1,256,256,2)
        yield (X_input,y_input)

model.fit_generator(GenerateInputs(X_train, y_train), epochs=50)

# Colorização das Imagens

In [ ]:
# Colorização
image_for_test = X_test[4]
output = model.predict(image_for_test)
output *= 128

# Junção de Canais de Cor com Luminosidade
coloredImage = np.zeros((256, 256, 3))
coloredImage[:,:,0] = image_for_test[0][:,:,0]
coloredImage[:,:,1:] = output[0]
coloredImage = lab2rgb(coloredImage)

In [ ]:
from matplotlib import pyplot as plt
plt.imshow(coloredImage)